# Step4: test

In [ ]:
def test_img(model_path, test_images):
  global patch
  test_patches = patch(test_images)

  loss_func = nn.MSELoss()
  new_model = Autoencoder4().to(device)
  new_model.load_state_dict(torch.load(model_path))
  new_model.eval()
  FP=0;TP=0;TN=0;FN=0;
  y_true = []
  y_pred = []
  img_idx=0
  for patch in test_patches:

    if img_idx<28:
        y_true.append(0) #정상
    else:
        y_true.append(1)

    defectness_list = []
    for patch_idx in range(256):
      one_patch = patch[patch_idx].view(1,1,64,64)
      one_patch = Variable(one_patch).cuda()
      pred = new_model(one_patch)
      loss = loss_func(pred, one_patch)
      defectness = patch_prediction(loss, 0.0028) # T / F값이 리턴됨

      defectness_list.append(defectness)
    image_defectness = image_prediction(3, defectness_list)

    if img_idx < 28:
      if image_defectness == True:
        y_pred.append(1)
      else:
        y_pred.append(0)
    else:
      if image_defectness == True:
        y_pred.append(1)
      else:
        y_pred.append(0)

    plt.imshow(np.transpose(test_images[img_idx].view(256,4096), (0, 1)), cmap="gray")
    plt.show()
    print("patch defect is {}.".format(sum(defectness_list)))
    if image_defectness == True:
      print("fabric #",img_idx," is DEFECT.")
    else:
      print("fabric #",img_idx," is nodefect.")
    print()
    print()
    img_idx +=1
  
  print("f1score = ",f1_score(y_true, y_pred))
  
  confusion_matrix2= confusion_matrix(y_true, y_pred)
  sns.heatmap(confusion_matrix2.T, annot=True,fmt='d',
              xticklabels=['nodefect','defect'], yticklabels=['nodefect','defect'],cmap="YlGnBu")
  plt.xlabel('true')
  plt.ylabel('predicted')
  plt.title('Confusion Matirx', fontsize=20)

In [ ]:
model_path = "./data/nocrop/model/finalepoch/Autoencoder4.pth" # 성능평가에서 제일 좋았던 걸로 선택
image_path =  "./data/nocrop/test/*.png"
test_images = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob(image_path))]  

In [ ]:
def patch(images):
  patch_size = 64
  global patch
  global patches
  for i in range(len(images)):
    images[i] = transforms.ToTensor()((images[i]))
    # unfold로 shape조절 --> view --> [patch개수, patch_size, patch_size]
    patches = images[i].data.unfold(1, patch_size, patch_size).unfold(2, patch_size, patch_size) 
    patches = patches.contiguous().view(-1, patch_size, patch_size) 

    if i == 0:
      patch = patches
    if i > 0:
      patches = torch.cat((patch, patches))
      patch = patches

  patches = patches.view(len(images), -1, patch_size, patch_size) 
  return patches

In [ ]:
test_img(model_path, test_images) # 3 0.0028